### What for?

If you have two related operations which you’d like to execute as a pair,
with a block of code in between. Context managers allow you to do specifically that.

### Examples

File reading, these two are equivalent codes:

In [10]:
file = open('README.md', 'r')
try:
    content = file.read()
finally:
    file.close() # don't forget to close it!

print(len(content))

2137


In [11]:
with open('README.md', 'r') as file:
    content = file.read()

print(len(content))

2137


In [ ]:
# Django atomic transaction for DB

from django.db import transaction

with transaction.atomic():
    # This code executes inside a transaction.
    obj = MyModel.objects.first()
    obj.my_field = 13
    obj.save()


### Implementing context manager:

In [34]:
import time

class MyTimer:
    DEBUG = False
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        if self.DEBUG:
            print(f'__enter__ ({self.code_name})')
        self.start_time = time.time()

    def __exit__(self, exception_type, exception_value, traceback):
        if self.DEBUG:
            print(f'__exit__ ({self.code_name})')
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')


a = 10
b = 100

with MyTimer("code1"):
    time.sleep(1)
    c = a ** b

Code: "code1" took 1.0028


In [35]:
with MyTimer("code2"):
    time.sleep(1)
    for i in range(a):
        with MyTimer(f"For loop [{i}]"):
            time.sleep(0.2)
            c = a ** a

Code: "For loop [0]" took 0.2013
Code: "For loop [1]" took 0.2049
Code: "For loop [2]" took 0.2001
Code: "For loop [3]" took 0.2040
Code: "For loop [4]" took 0.2031
Code: "For loop [5]" took 0.2037
Code: "For loop [6]" took 0.2001
Code: "For loop [7]" took 0.2009
Code: "For loop [8]" took 0.2052
Code: "For loop [9]" took 0.2002
Code: "code2" took 3.0277


In [36]:
my_timer = MyTimer("code3")
my_timer.DEBUG = True

with my_timer:
    time.sleep(1)

__enter__ (code3)
__exit__ (code3)
Code: "code3" took 1.0006


`__enter__`  can return the object that will be used in `with <manager> as <return_from_enter>:`

In [ ]:
class MyTimer2:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return 3

    def __exit__(self, exception_type, exception_value, traceback):
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')

my_timer2 = MyTimer2('code4')
with my_timer2 as something:
    print(something)  # something is `3`
    time.sleep(1)


In [39]:
class MyTimer3:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None
        self.betweens = []

    def __enter__(self):
        self.start_time = time.time()
        return self

    def between(self):
        self.betweens.append(time.time() - self.start_time)

    def __exit__(self, exception_type, exception_value, traceback):
        betweens = ', '.join(f'{x:.3f}' for x in self.betweens)
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}. Betweens: {betweens}')


with MyTimer3('code5') as timer:
    print(timer)  # something is `3`
    time.sleep(1)
    timer.between()
    time.sleep(0.2)
    timer.between()
    time.sleep(0.2)


Code: "code4" took 1.4117. Betweens: 1.003, 1.208


In [ ]:
class MyTimer4:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        print("Will __exit__ be executed?")
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')


with MyTimer4('code6') as timer:
    time.sleep(0.2)
    raise Exception("Ooops")
    print('Will it reach this code?')

print('outside of with')

`__exit__` should return boolean, whether exception is handled by it context manager or not

In [57]:
class MyTimer5:
    def __init__(self, code_name):
        self.code_name = code_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        print("Will __exit__ be executed?")
        print(f'Code: "{self.code_name}" took {time.time() - self.start_time:.4f}')
        print(exception_type, exception_value)
        if isinstance(exception_value, KeyError):
            return True

In [58]:
with MyTimer5('code6') as timer:
    time.sleep(0.2)
    raise KeyError("Ooops")
    print('Will it reach this code?')

print('outside of with')

Will __exit__ be executed?
Code: "code6" took 0.2026
<class 'KeyError'> 'Ooops'
outside of with


In [59]:
with MyTimer5('code6') as timer:
    time.sleep(0.2)
    raise Exception("Ooops")
    print('Will it reach this code?')

print('outside of with')


Will __exit__ be executed?
Code: "code6" took 0.2053
<class 'Exception'> Ooops


Exception: Ooops

### Implementing context manager as function

Classic example

In [70]:
from contextlib import contextmanager

@contextmanager
def my_timer(filename):
    f = open(filename, 'r')
    try:
        yield f # you can yield nothing with `yield` if you yield something it can be used with "as ..."
    finally:
        f.close()


with my_timer('README.md') as file:
    print(f'{file=}')
    content = file.read()
    print(len(content))

file=<_io.TextIOWrapper name='README.md' mode='r' encoding='UTF-8'>
2137


Timer example

In [71]:
from contextlib import contextmanager

@contextmanager
def my_timer(code_name):
    start_time = time.time()
    try:
        yield
    finally:
        print(f'Code: "{code_name}" took {time.time() - start_time:.4f}')

with my_timer('code 7'):
    time.sleep(1)

Code: "code 7" took 1.0020
